In [6]:
import pandas as pd
import numpy as np
import os

NOTEBOOK_DIR = os.getcwd()
project_dir = os.path.dirname(NOTEBOOK_DIR)
apps_dir = os.path.join(project_dir, 'apps')
home_dir = os.path.join(apps_dir, 'home')
static_dir = os.path.join(apps_dir, 'static')
DATA_DIR = os.path.join(static_dir, 'assets', 'data')

print('notebook_dir:', NOTEBOOK_DIR)
print('project_dir:', project_dir)
print('apps_dir:', apps_dir)
print('home_dir:', home_dir)
print('static_dir:', static_dir)
print('DATA_DIR:', DATA_DIR)

notebook_dir: c:\Coding\EcoThrift\notebooks
project_dir: c:\Coding\EcoThrift
apps_dir: c:\Coding\EcoThrift\apps
home_dir: c:\Coding\EcoThrift\apps\home
static_dir: c:\Coding\EcoThrift\apps\static
DATA_DIR: c:\Coding\EcoThrift\apps\static\assets\data


### Load Site design Excel file and export as pickel file for faster operation

In [7]:
site_design_excel = os.path.join(NOTEBOOK_DIR, 'field_design.xlsm')

SD_MODELS_FN = os.path.join(DATA_DIR, 'models.pkl')
SD_MODEL_FIELDS_FN = os.path.join(DATA_DIR, 'model_fields.pkl')

SD_FORMS_FN = os.path.join(DATA_DIR, 'forms.pkl')
SD_FORM_FIELDS_FN = os.path.join(DATA_DIR, 'form_fields.pkl')
SD_META_FORMS_FN = os.path.join(DATA_DIR, 'meta_forms.pkl')

SD_TABLES_FN = os.path.join(DATA_DIR, 'tables.pkl')
SD_TABLE_FIELDS_FN = os.path.join(DATA_DIR, 'table_fields.pkl')

SD_PAGES_FN = os.path.join(DATA_DIR, 'pages.pkl')
SD_PAGE_ELEMENTS_FN = os.path.join(DATA_DIR, 'page_elements.pkl')


SD_MODELS = ['Models', site_design_excel, 'models', [], SD_MODELS_FN]
SD_MODEL_FIELDS = ['Model Fields', site_design_excel, 'model_fields', [], SD_MODEL_FIELDS_FN]

SD_FORMS = ['Forms', site_design_excel, 'forms', [], SD_FORMS_FN]
SD_FORM_FIELDS = ['Form Fields', site_design_excel, 'form_fields', ['Row','Column', 'Size'], SD_FORM_FIELDS_FN]
SD_META_FORMS = ['Meta Forms', site_design_excel, 'meta_forms', ['Order'], SD_META_FORMS_FN]


SD_TABLES = ['Tables', site_design_excel, 'tables', [], SD_TABLES_FN]
SD_TABLE_FIELDS = ['Table Fields', site_design_excel, 'table_fields', ['Order'], SD_TABLE_FIELDS_FN]

SD_PAGES = ['Pages', site_design_excel, 'pages', [], SD_PAGES_FN]
SD_PAGE_ELEMENTS = ['Page Elements', site_design_excel, 'page_elements', ['Row','Column'], SD_PAGE_ELEMENTS_FN]

def convert_cols(df, int_cols=[]):
    cols = df.columns.tolist()
    str_cols = [c for c in cols if c not in int_cols]
    dt_dct = {c:np.dtype('i1') for c in int_cols}
    dt_dct.update({c:np.dtype('str') for c in str_cols})
    df[int_cols] = df[int_cols].fillna(-1)
    df[str_cols] = df[str_cols].fillna('')
    df = df.astype(dt_dct)
    return df

def export_form_pickel(args):
    df = pd.read_excel(args[1], sheet_name=args[2])
    df = convert_cols(df, args[3])
    df.to_pickle(args[4])
    print('Exported PKL: ', args[0], df.shape)
    return

def export_pickels():
    export_form_pickel(SD_MODELS)
    export_form_pickel(SD_FORMS)
    export_form_pickel(SD_MODEL_FIELDS)
    export_form_pickel(SD_FORM_FIELDS)
    export_form_pickel(SD_META_FORMS)
    export_form_pickel(SD_TABLES)
    export_form_pickel(SD_TABLE_FIELDS)
    export_form_pickel(SD_PAGES)
    export_form_pickel(SD_PAGE_ELEMENTS)
    return

export_pickels()

Exported PKL:  Models (17, 5)
Exported PKL:  Forms (13, 3)
Exported PKL:  Model Fields (102, 6)
Exported PKL:  Form Fields (97, 12)
Exported PKL:  Meta Forms (22, 11)
Exported PKL:  Tables (9, 6)
Exported PKL:  Table Fields (109, 10)
Exported PKL:  Pages (29, 7)
Exported PKL:  Page Elements (28, 10)


### Print models.py code generated from site design

In [9]:
def model_str(model_list=None, copy_to_clipboard = True):    
    export_pickels()
    include_dict_code = False
    if model_list is None:
        model_list = list(pd.read_pickle(SD_MODELS_FN).Name)
        include_dict_code = True
    code_list = ['']
    model_df = pd.read_pickle(SD_MODELS_FN)
    field_df = pd.read_pickle(SD_MODEL_FIELDS_FN)    
    for nm in model_list:
        m = model_df[model_df.Name == nm].iloc[0]
        fields = "\n".join([f'    {c}'for c in field_df[field_df.Model == nm].Code.values])
        if fields == '':
            fields = "    # No Fields Defined"
        code = "\n".join([
            f"\nclass {m.Name}({m.Extends}):",
            f"    model_name, _pk_ = '{m.Name}', '{m.PK}'",
            f"    model_design = MODEL_DESIGN[model_name]",            
            f"    ",
            f"{fields}",
            f"    ",
            f'    def __str__(self):',
            f"        return f'{m.String}'"
        ])
        code_list += [code]

    if include_dict_code:
        models = "{" + ", ".join([f"'{nm}':{nm}"for nm in model_list]) + "}"
        code_list += [f"\nMODELS = {models}"]

    s = "\n".join(code_list)

    if copy_to_clipboard:
        pd.DataFrame([s]).to_clipboard(index=False, header=False, quotechar='\t')
    return s

s = model_str()

Exported PKL:  Models (17, 5)
Exported PKL:  Forms (13, 3)
Exported PKL:  Model Fields (102, 6)
Exported PKL:  Form Fields (97, 12)
Exported PKL:  Meta Forms (22, 11)
Exported PKL:  Tables (9, 6)
Exported PKL:  Table Fields (109, 10)
Exported PKL:  Pages (29, 7)
Exported PKL:  Page Elements (28, 10)


In [5]:
def form_str(form_list=None):    
    export_pickels()

    form_df = pd.read_pickle(SD_FORMS_FN)
    form_fields_df = pd.read_pickle(SD_FORM_FIELDS_FN)

    include_dict_code = False
    if form_list is None:
        form_list = list(form_df.Name)
        include_dict_code = True

    code_list = ['']    
    exclude_list = []

    for nm in form_list:
        if nm not in exclude_list:
            f = form_df[form_df.Name == nm].iloc[0]
            fields = "', '".join(form_fields_df[form_fields_df.Form == f.Name].Name.tolist())
            code = "\n".join([
                f"\nclass {f.Name}({f.Extends}):",
                f"    form_name = '{f.Name}'",
                f"    form_design = FORM_DESIGN['{f.Name}']",
                f"    field_struct, field_attrs = GET_FORM_FIELDS(form_design)",
                f"    model_class = {f.Model}",
                f"    model_name = '{f.Model}'",
                f"    class Meta:",
                f"      model = {f.Model}",
                f"      fields = ['{fields}']"
            ])
            code_list += [code]

    if include_dict_code:
        forms = "{" + ", ".join([f"'{nm}':{nm}"for nm in form_list]) + "}"
        code_list += [f"\nFORMS = {forms}"]

    s = "\n".join(code_list)
    
    pd.DataFrame([s]).to_clipboard(index=False, header=False, quotechar='\t')
    return

form_str()


Exported PKL:  Models (17, 5)
Exported PKL:  Forms (13, 3)
Exported PKL:  Model Fields (102, 6)
Exported PKL:  Form Fields (97, 12)
Exported PKL:  Meta Forms (22, 11)
Exported PKL:  Tables (9, 6)
Exported PKL:  Table Fields (109, 10)
Exported PKL:  Pages (29, 7)
Exported PKL:  Page Elements (28, 10)


In [197]:
l = [('asdf', 'asdf')]
dict(l)

{'asdf': 'asdf'}

In [42]:
def sidebar():
    sidebar = []
    menu, menu2, sub, sub2 = None, None, None, None
    thisMenu, thisSub = [], []
    df = pd.read_excel(site_design_excel, sheet_name='sidebar')
    df = convert_cols(df)

    menu_cols = ['Menu','MenuId', 'Icon']
    sub_cols = ['SubMenu', 'SubId', 'Type','Text','Mini', 'SubMini']
    text_cols = ['Text','Mini','HREF']    

    for _, r in df.iterrows():
        if sub != r['SubMenu']:
            if sub:
                thisMenu += [(sub2, thisSub)]
            sub, sub2, thisSub = r['SubMenu'], dict(r[sub_cols]), []
        if menu != r['Menu']:
            if menu:
                sidebar += [(menu2, thisMenu)]
            menu, menu2, thisMenu = r['Menu'], dict(r[menu_cols]), []
        thisSub += [dict(r[text_cols])]
        
    thisMenu += [(sub2, thisSub)]
    sidebar += [(menu2, thisMenu)]

    return sidebar
    
print(sidebar())

#display(sidebar())

[({'Menu': 'Home', 'MenuId': 'HomePages', 'Icon': 'fas fa-couch'}, [({'SubMenu': 'Dashboard', 'SubId': 'HomeDashboardPages', 'Type': '0', 'Text': 'Dashboard', 'Mini': 'D', 'SubMini': 'H'}, [{'Text': 'Dashboard', 'Mini': 'D', 'HREF': 'Dashboard'}]), ({'SubMenu': 'Profile', 'SubId': 'HomeProfilePages', 'Type': '0', 'Text': 'Profile', 'Mini': 'P', 'SubMini': 'H'}, [{'Text': 'Profile', 'Mini': 'P', 'HREF': 'MyProfile'}]), ({'SubMenu': 'TimeClock', 'SubId': 'HomeTimeClockPages', 'Type': '0', 'Text': 'TimeClock', 'Mini': 'T', 'SubMini': 'H'}, [{'Text': 'TimeClock', 'Mini': 'T', 'HREF': ''}]), ({'SubMenu': 'Calendar', 'SubId': 'HomeCalendarPages', 'Type': '0', 'Text': 'Calendar', 'Mini': 'C', 'SubMini': 'H'}, [{'Text': 'Calendar', 'Mini': 'C', 'HREF': ''}]), ({'SubMenu': 'Events', 'SubId': 'HomeEventsPages', 'Type': '1', 'Text': 'New Event', 'Mini': 'N', 'SubMini': 'H'}, [{'Text': 'New Event', 'Mini': 'N', 'HREF': ''}, {'Text': 'View Events', 'Mini': 'V', 'HREF': ''}])]), ({'Menu': 'Operation

In [41]:
nm = 'ProfileForm'
META_FORMS = pd.read_pickle(SD_META_FORMS_FN).sort_values(by=['Order'])
COLS = ['Forms','Template','Header','SubHeader','Button','Instance','Redirect_Valid','Redirect_Invalid','Delete']
df = META_FORMS[META_FORMS.Name == nm]
template= df.Template.iloc[0]
vals = {c:df[c].tolist() for c in COLS}
vals2 = [dict(i) for _, i in df[COLS].iterrows()]
vals2

[{'Forms': 'UserForm',
  'Template': 'forms/pill-forms.html',
  'Header': 'My User',
  'SubHeader': 'You can edite your User profile below',
  'Button': 'Edit',
  'Instance': 'pk',
  'Redirect_Valid': 'MyProfile',
  'Redirect_Invalid': 'MyProfile',
  'Delete': ''},
 {'Forms': 'ShopperForm',
  'Template': 'forms/pill-forms.html',
  'Header': 'My Shopper',
  'SubHeader': 'You can edite your Shopper profile below',
  'Button': 'Edit',
  'Instance': 'pk',
  'Redirect_Valid': 'MyProfile',
  'Redirect_Invalid': 'MyProfile',
  'Delete': ''},
 {'Forms': 'ConsignorForm',
  'Template': 'forms/pill-forms.html',
  'Header': 'My Consignor',
  'SubHeader': 'You can edite your Consignor profile below',
  'Button': 'Edit',
  'Instance': 'pk',
  'Redirect_Valid': 'MyProfile',
  'Redirect_Invalid': 'MyProfile',
  'Delete': ''},
 {'Forms': 'EmployeeForm',
  'Template': 'forms/pill-forms.html',
  'Header': 'My Employee',
  'SubHeader': 'You can edite your Employee profile below',
  'Button': 'Edit',
  'In

In [46]:
cur_dict = {
    'cur_user': 'request.user',
    'cur_shp':  'request.user.extendeduser.shopper',
    'cur_con':  'request.user.extendeduser.consignor',
    'cur_emp':  'request.user.extendeduser.employee'
}

pk = 'cur_user'

if pk in cur_dict:
    print(cur_dict[pk])
else:
    print(pk)


request.user
